In [ ]:
#loading libraries:
import numpy as np
import pandas as pd


def strat_sample_city_star(
    df,
    n=500,
    city_col="city",
    star_col="star_count",
    top_n_cities=15,
    seed=42,
):
    rng = np.random.default_rng(seed)
    d = df.copy()

    # Basic cleanup for the relevant columns
    d[city_col] = d[city_col].astype(str).str.strip()
    d[star_col] = pd.to_numeric(d[star_col], errors="coerce")

    # Keep only rows where both city and star_count exist
    d = d.dropna(subset=[city_col, star_col]).copy()
    if d.empty:
        raise ValueError("No rows left after dropping missing city/star values.")

    d[star_col] = d[star_col].astype(int)

    # Top cities as separate buckets, everything else = Other_Cities
    top_cities = d[city_col].value_counts().head(top_n_cities).index
    d["city_bucket"] = np.where(d[city_col].isin(top_cities), d[city_col], "Other_Cities")

    # Star groups: 0/1/2 and everything 3+ merged into one group
    d["star_group"] = np.where(d[star_col] >= 3, 3, d[star_col])

    # Stratum = (city bucket, star group)
    d["stratum"] = list(zip(d["city_bucket"], d["star_group"]))

    # Stratum sizes
    counts = d.groupby("stratum").size()
    N = int(counts.sum())
    if N == 0:
        raise ValueError("No valid observations available.")

    # Proportional allocation
    n_prop = (counts / N) * n
    n_h = np.floor(n_prop).astype(int)

    # Distribute rounding remainder (largest fractional parts first)
    frac = (n_prop - np.floor(n_prop)).sort_values(ascending=False)
    rest = int(n - n_h.sum())

    if rest > 0:
        for s in frac.index:
            if rest == 0:
                break
            if n_h.loc[s] < counts.loc[s]:
                n_h.loc[s] += 1
                rest -= 1
    elif rest < 0:
        frac2 = (n_prop - np.floor(n_prop)).sort_values(ascending=True)
        for s in frac2.index:
            if rest == 0:
                break
            if n_h.loc[s] > 0:
                n_h.loc[s] -= 1
                rest += 1

    # One random number per row:
    d["_u"] = rng.random(len(d))

    # Sample inside each stratum
    parts = []
    for s, k in n_h.items():
        k = int(k)
        if k <= 0:
            continue
        tmp = d[d["stratum"] == s].nsmallest(k, "_u")
        parts.append(tmp)

    if parts:
        sample = pd.concat(parts, axis=0).copy()
    else:
        sample = d.iloc[0:0].copy()

    sample = sample.drop(columns=["_u"])

    # Enforce exactly n rows 
    if len(sample) > n:
        sample = sample.sample(n=n, random_state=seed)
    elif len(sample) < n:
        missing = n - len(sample)
        rest_df = d.loc[~d.index.isin(sample.index)].copy()
        if not rest_df.empty:
            extra = rest_df.sample(n=min(missing, len(rest_df)), random_state=seed).drop(columns=["_u"])
            sample = pd.concat([sample, extra], axis=0)

        if len(sample) > n:
            sample = sample.sample(n=n, random_state=seed)

    sample = sample.reset_index(drop=True)

    # Allocation table for documentation / reporting
    alloc = (
        pd.DataFrame({"stratum": counts.index, "N_h": counts.values})
        .merge(pd.DataFrame({"stratum": n_h.index, "n_h": n_h.values}), on="stratum", how="left")
        .fillna({"n_h": 0})
    )
    alloc["n_h"] = alloc["n_h"].astype(int)
    alloc[["city_bucket", "star_group"]] = pd.DataFrame(alloc["stratum"].tolist(), index=alloc.index)
    alloc = alloc.drop(columns=["stratum"]).sort_values(["n_h", "N_h"], ascending=False).reset_index(drop=True)

    return sample, alloc


if __name__ == "__main__":
    df = pd.read_excel(
        r"\PDA_DATA\Restaurants_Data.xlsx",
        sheet_name="Data_Filtered_Primary",
    )

    sample_500, alloc_table = strat_sample_city_star(
        df,
        n=500,
        city_col="city",
        star_col="star_count",
        top_n_cities=15,
        seed=42,
    )

    sample_500.to_excel("sample_500_city_star.xlsx", index=False)
    alloc_table.to_csv("allocation_city_star.csv", index=False)